In [1]:
# !pip install --upgrade xgboost

In [2]:
import pandas as pd
# from pandas_profiling import ProfileReport as PR
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
import plotly.express as px
import os, sys

from ydata_profiling import ProfileReport

%matplotlib inline

In [3]:
working_directory = os.getcwd()

In [4]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

sample_submission = pd.read_csv("sample_submission.csv")

In [5]:
train_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


In [6]:
train_df.tail()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.0,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.0,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.0,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.0,1,0.0,1.0,71173.03,0
165033,165033,15732798,Ulyanov,850,France,Male,31.0,1,0.0,1,1.0,0.0,61581.79,1


In [7]:
print(f"Shape of training data set - {train_df.shape}")
print(f"Shape of test data set - {test_df.shape}")

Shape of training data set - (165034, 14)
Shape of test data set - (110023, 13)


In [8]:
print(f"Number of customers training - {train_df.CustomerId.nunique()}")
print(f"Number of customers testing - {test_df.CustomerId.nunique()}")

Number of customers training - 23221
Number of customers testing - 19698


In [9]:
test_df.head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,165034,15773898,Lucchese,586,France,Female,23.0,2,0.00,2,0.0,1.0,160976.75
1,165035,15782418,Nott,683,France,Female,46.0,2,0.00,1,1.0,0.0,72549.27
2,165036,15807120,K?,656,France,Female,34.0,7,0.00,2,1.0,0.0,138882.09
3,165037,15808905,O'Donnell,681,France,Male,36.0,8,0.00,1,1.0,0.0,113931.57
4,165038,15607314,Higgins,752,Germany,Male,38.0,10,121263.62,1,1.0,0.0,139431.00


In [10]:
train_df[train_df.CustomerId == 15674932].head()

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.0,2,1.0,0.0,181449.97,0
671,671,15674932,Genovesi,619,France,Female,37.0,8,0.0,2,1.0,0.0,162858.29,1
4134,4134,15674932,K'ung,667,France,Female,43.0,9,0.0,2,1.0,1.0,119882.70,0
4768,4768,15674932,Williamson,658,Spain,Female,43.0,6,0.0,1,1.0,0.0,16301.91,1
19161,19161,15674932,Norman,670,Spain,Female,50.0,7,0.0,1,1.0,0.0,177528.92,1


Seems CustomerID is not unique for customers and does not bring any significant information so dropping the column

In [11]:
train_df.drop(columns={"CustomerId"}, inplace=True)
test_df.drop(columns={"CustomerId"}, inplace=True)

In [12]:
train_df[train_df.Surname == "Okwudilichukwu"].head()

,id,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
704,704,Okwudilichukwu,653,Germany,Male,41.0,2,92350.37,1,1.0,1.0,136435.75,0
2189,2189,Okwudilichukwu,790,France,Male,42.0,1,0.00,1,0.0,1.0,173948.30,0
3335,3335,Okwudilichukwu,703,France,Female,38.0,5,0.00,2,1.0,0.0,131906.44,0
3720,3720,Okwudilichukwu,604,France,Male,35.0,2,0.00,2,1.0,1.0,114514.96,0


In [13]:
train_df.groupby(["Surname"]).id.count()

Surname
Abazu        1
Abbie        4
Abbott      24
Abdullah     1
Abdulov     12
            ..
Zubarev      7
Zubareva     1
Zuev        19
Zuyev       58
Zuyeva      60
Name: id, Length: 2797, dtype: int64

Seems Surname is also not unique for customers and does not bring any significant information so dropping the column

In [14]:
train_df.drop(columns={"Surname"}, inplace=True)
test_df.drop(columns={"Surname"}, inplace=True)

In [15]:
train_df.head()

,id,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0


## Analyzing individual columns

In [16]:
# Changing the data type of id
train_df.id = train_df.id.astype("category")
test_df.id = test_df.id.astype("category")

In [17]:
data_profile = ProfileReport(train_df, title = "Data Profile")

In [18]:
data_profile.to_file("data_summary.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_df.describe()

<br> id - categorical (unique id) - to drop
<br> Geography - 3 Locations (France, Span Germany)
<br> Gender - Male, Female
<br> Tenure - 0 to 10 (11 Values)
<br> Balance - Balance held by customer (continuous variable)
<br> Number of Products - Continuous (1, 2, 3, 4)
<br> Has Credit Card - Categorical (1,0)
<br> Is Active Member - Categorical (1,0)
<br> Estimated Salary - Continuous
<br> Exited - Categorical (1,0)
<br> Credit Score - Continuous
<br> Age - Continuous

In [ ]:
# Changing the data type of id
train_df.id = train_df.id.astype("category")
test_df.id = test_df.id.astype("category")

In [ ]:
fig = px.histogram(train_df, x="CreditScore", histnorm='probability density', color="Exited")
fig.show()

In [ ]:
fig = px.histogram(train_df, x="Balance", histnorm='probability density', color="Exited")
fig.show()

In [ ]:
print(train_df.groupby(by="Exited").id.count())
# sns.countplot(x = "Survived", data=train_df)
# plt.show()

In [ ]:
fig = px.box(train_df, x="Tenure", y="Balance", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Gender", y="Balance", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Gender", y="Age", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Gender", y="EstimatedSalary", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Geography", y="CreditScore", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Geography", y="Age", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Geography", y="Balance", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Geography", y="EstimatedSalary", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Tenure", y="CreditScore", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Tenure", y="Age", color="Exited")
fig.show()

In [ ]:
fig = px.box(train_df, x="Tenure", y="Balance", color="Exited")
fig.show()

In [ ]:
print(train_df.Age.mean())
print(train_df[train_df.Tenure == 0].Age.mean())
print(train_df[train_df.Tenure == 1].Age.mean())
print(train_df[train_df.Tenure == 2].Age.mean())
print(train_df[train_df.Tenure == 3].Age.mean())
print(train_df[train_df.Tenure == 4].Age.mean())
print(train_df[train_df.Tenure == 5].Age.mean())

In [ ]:
geo = train_df.groupby(["Geography","IsActiveMember","NumOfProducts","Exited"]).Exited.count()
# geo["%"] = geo.
# print(geo)
print(geo.groupby(level=[0,1,2]).apply(lambda x:100 * x / float(x.sum())))

In [ ]:
fig = px.box(train_df, x="Tenure", y="Balance", color="Exited")
fig.show()

<br> Age
<br> Balance
<br> Credit Score
<br> EstimatedSalary
<br> Gender - OnehotEncoding
<br> Geography - OnHOtEncoding
<br> ~~Has Credit Card-~~
<br> IsActiveMember - OnehotEncoding
<br> NumOfProducts - OnehotEncoding
<br> ~~Tenur~~ 

In [19]:
_train_df = train_df.drop(columns=["HasCrCard","Tenure"])

In [20]:
_train_df.columns

Index(['id', 'CreditScore', 'Geography', 'Gender', 'Age', 'Balance',
       'NumOfProducts', 'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [21]:
_train_df.head()

,id,CreditScore,Geography,Gender,Age,Balance,NumOfProducts,IsActiveMember,EstimatedSalary,Exited
0,0,668,France,Male,33.0,0.00,2,0.0,181449.97,0
1,1,627,France,Male,33.0,0.00,2,1.0,49503.50,0
2,2,678,France,Male,40.0,0.00,2,0.0,184866.69,0
3,3,581,France,Male,34.0,148882.54,1,1.0,84560.88,0
4,4,716,Spain,Male,33.0,0.00,2,1.0,15068.83,0


In [22]:
def get_features(table):
    df = table.copy()
    df["Gender"] = df["Gender"].map({"Male": 0, "Female": 1})
    df['Gender'] = df['Gender'].astype('uint8')
    # create dummy variables for categorical features
    geo_dummies = pd.get_dummies(df.Geography, prefix="geo")
    prod_dummies = pd.get_dummies(df.NumOfProducts, prefix="prod")
    # concatenate dummy columns with main dataset
    table_dummies = pd.concat([df, geo_dummies, prod_dummies], axis=1)

    # drop categorical fields
    table_dummies.drop(['id','Geography', 'NumOfProducts'], axis=1, inplace=True)
    return table_dummies

In [23]:
train = get_features(_train_df)

In [24]:
train.head()

,CreditScore,Gender,Age,Balance,IsActiveMember,EstimatedSalary,Exited,geo_France,geo_Germany,geo_Spain,prod_1,prod_2,prod_3,prod_4
0,668,0,33.0,0.00,0.0,181449.97,0,1,0,0,0,1,0,0
1,627,0,33.0,0.00,1.0,49503.50,0,1,0,0,0,1,0,0
2,678,0,40.0,0.00,0.0,184866.69,0,1,0,0,0,1,0,0
3,581,0,34.0,148882.54,1.0,84560.88,0,1,0,0,1,0,0,0
4,716,0,33.0,0.00,1.0,15068.83,0,0,0,1,0,1,0,0


In [25]:
x_train = train.drop('Exited', axis=1) 
y_train = train['Exited']

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
# from skopt import gp_minimize
# from skopt.space import Real, Integer
from functools import partial

In [27]:
train_x, test_x, train_y, test_y = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)

In [28]:
def model(X,Y,x,y):

#     Logistinc Regression
    log = LogisticRegression(random_state=0)
    log.fit(X,Y)
    # log.predict(x)
    
#     Decision Tree
    des_tree = DecisionTreeClassifier(criterion='entropy',random_state=0)
    des_tree.fit(X,Y)
    # des_tree.predict(x)
    
#     Knearestneighbour
    knn = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p =2)
    knn.fit(X,Y)
    # knn.predict(x)
    
#     Random Forrest Model
    forrest = RandomForestClassifier(random_state=0)
    forrest.fit(X,Y)
    # forrest.predict(x)
    
# Using SVC method of svm class to use Support Vector Machine Algorithm
    svc_lin = SVC(kernel = 'linear', random_state = 0)
    svc_lin.fit(X, Y)
    # svc_lin.predict(x)

#Using SVC method of svm class to use Kernel SVM Algorithm
    svc_rbf = SVC(kernel = 'rbf', random_state = 0)
    svc_rbf.fit(X, Y)
    # svc_rbf.predict(x)

#Using GaussianNB method of naïve_bayes class to use Naïve Bayes Algorithm
    gauss = GaussianNB()
    gauss.fit(X, Y)
    # gauss.predict(x)
    
#     XGBoost Classifier
    xg = XGBClassifier()
    xg.fit(X,Y)
    # xg.predict(x)
    
    print('Logistic Regression Training Accuracy {}, Validation Accuracy {}:'.format(log.score(X, Y), log.score(x,y)))
    print('K Nearest Neighbor Training Accuracy {}: Validation Accuracy {}'.format(knn.score(X, Y), knn.score(x,y)))
    print('Support Vector Machine (Linear Classifier) Training Accuracy {}: Validation Accuracy {}'.format(svc_lin.score(X, Y), svc_lin.score(x,y)))
    print('Support Vector Machine (RBF Classifier) Training Accuracy {}: Validation Accuracy {}'.format(svc_rbf.score(X, Y),svc_rbf.score(x,y) ))
    print('Gaussian Naive Bayes Training Accuracy {}: Validation Accuracy {}'.format(gauss.score(X, Y), gauss.score(x,y)))
    print('Decision Tree Classifier Training Accuracy {}: Validation Accuracy {}'.format(des_tree.score(X, Y),des_tree.score(x,y) ))
    print('Random Forest Classifier Training Accuracy {}: Validation Accuracy {}'.format(forrest.score(X, Y), forrest.score(x,y)))  
    print('XGBoost Classifier Training Accuracy {}: Validation Accuracy {}'.format(xg.score(X, Y), xg.score(x,y)))
    
    return log, des_tree, forrest, svc_lin, svc_rbf, gauss, xg

In [29]:
models = model(train_x, train_y, test_x, test_y)

In [ ]:
logistic_model = models[0]
decision_tree = model[1]
random_forest = model[2]
svm_l = model[3]
svm_rbf = model[4]
naive_bayes = model[5]
xgb = model[6]